In [ ]:
import pandas as pd
train=pd.read_csv('../input/forecast-energy-consumption/training_data.csv')
train.head()

In [ ]:
train.info()

In [ ]:
test=pd.read_csv('../input/forecast-energy-consumption/testing_data.csv')
test.head()

In [ ]:
train.Date=pd.to_datetime(train.Date, infer_datetime_format=True)
train.info()

In [ ]:
test.Date=pd.to_datetime(test.Date, infer_datetime_format=True)
test.info()

In [ ]:
!pip install fast_ml
!pip install flaml
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
from fast_ml.feature_engineering import FeatureEngineering_DateTime
dt_fe = FeatureEngineering_DateTime()
dt_fe.fit(train, datetime_variables=['Date'])
train = dt_fe.transform(train)
train.head()

In [ ]:
dt_fe.fit(test, datetime_variables=['Date'])
test = dt_fe.transform(test)
test.head()

In [ ]:
nunique_train=train.nunique().reset_index()
remove_col=nunique_train[(nunique_train[0]==len(train)) | (nunique_train[0]==0) | (nunique_train[0]==1) ]['index'].tolist()
remove_col[:5]

In [ ]:
pd.set_option('display.max_columns', None)
train=train.drop(remove_col,axis=1)
train.head()

In [ ]:
test=test.drop(remove_col,axis=1)
test.head()

In [ ]:
train=train.drop(['Date:time'],axis=1)
train.info()

In [ ]:
test=test.drop(['Date:time'],axis=1)
test.info()

In [ ]:
train['Date:day_part'].unique()

In [ ]:
train['Date:day_part'].value_counts()

In [ ]:
import numpy as np
train['Date:day_part'] = train['Date:day_part'].fillna(value=np.nan)
train['Date:day_part'].unique()

In [ ]:
test['Date:day_part'] = test['Date:day_part'].fillna(value=np.nan)
test['Date:day_part'].unique()

In [ ]:
round(train.isnull().sum()*100/len(train),2).sort_values(ascending=False)

In [ ]:
train[train.isnull().any(axis=1)]

In [ ]:
train['Date:day_part'] = train['Date:day_part'].fillna(value='midnight')
round(train.isnull().sum()*100/len(train),2).sort_values(ascending=False)[:5]

In [ ]:
test['Date:day_part'] = test['Date:day_part'].fillna(value='midnight')
round(test.isnull().sum()*100/len(test),2).sort_values(ascending=False)[:5]

In [ ]:
train['Date:day_part']=train['Date:day_part'].replace({'midnight':0,'dawn':1, 'early morning':2, 'late morning':3, 'noon':4,
                                                       'afternoon':5,'evening':6, 'night':7 })
      
train['Date:day_part'].value_counts()

In [ ]:
test['Date:day_part']=test['Date:day_part'].replace({'midnight':0,'dawn':1, 'early morning':2, 'late morning':3, 'noon':4,
                                                       'afternoon':5,'evening':6, 'night':7 })
      
test['Date:day_part'].value_counts()

In [ ]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [ ]:
train, NAlist = reduce_mem_usage(train)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

In [ ]:
test, NAlist = reduce_mem_usage(test)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

In [ ]:
import re
train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
train.head()

In [ ]:
test = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test.head()

In [ ]:
y = train.pop('Power')
X = train

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

In [ ]:
from flaml import AutoML
automl = AutoML()

In [ ]:
from flaml import AutoML
automl = AutoML()
automl.fit(X_train, y_train, task="regression",metric='rmse',time_budget=900)

In [ ]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best rmse on validation data: {0:.4g}'.format(automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

In [ ]:
from flaml.ml import sklearn_metric_loss_score
print('train rmse', '=', sklearn_metric_loss_score('rmse', automl.predict(X_train), y_train))
print(' test rmse', '=', sklearn_metric_loss_score('rmse', automl.predict(X_test), y_test))

In [ ]:
y_pred = automl.predict(test)
y_pred[:5]

In [ ]:
sol = pd.DataFrame(y_pred,columns=['Power'])
sol.head()

In [ ]:
sol.to_csv('./rmse.csv',index=False)